In [64]:
import pandas as pd
import numpy as np
import itertools
import snscrape.modules.twitter as sntwitter

from langchain import LLMChain
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

import os

In [32]:
api_key = ""

llm = OpenAI(temperature=0, openai_api_key=api_key,
             model_name="gpt-3.5-turbo", max_tokens=256)

In [59]:
import re

tweet_id = 1629481601116811265

tweets = sntwitter.TwitterTweetScraper(tweet_id, mode="recurse").get_items()

tweets


<generator object TwitterTweetScraper.get_items at 0x000001D66A736340>

In [63]:
username = "slowburninfuse"
getdata = sntwitter.TwitterUserScraper(username).get_items()
getdata_dict = list(getdata)

getdata_dict

KeyboardInterrupt: 

In [60]:
tweets

<generator object TwitterTweetScraper.get_items at 0x000001D66A736340>

In [49]:
tweets_dict = list(tweets)

tweets_dict

[]

In [17]:
import datetime
import itertools
import pandas as pd
import pytz
import snscrape.modules.twitter as sntwitter

def get_tweets(entities, start_date, end_date, num_tweets):
    """
    Scrape tweets for a list of entities within a specified date range and return a Pandas DataFrame.

    Parameters:
    ----------
    entities : list
        A list of Twitter entities (e.g., usernames, hashtags) to search for.
    start_date : str
        The start date for the search in YYYY-MM-DD format.
    end_date : str
        The end date for the search in YYYY-MM-DD format.
    num_tweets : int
        The number of tweets to scrape for each entity.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the scraped tweets for all entities.
        Columns: entity, url, postedTime, tweetText, id, username, retweetCount, likeCount, quoteCount, viewCount,
        scrapedTime, date, month, year
    """
    all_tweets_df = pd.DataFrame()

    for entity in entities:
        # Build the Twitter search query
        query = f'"{entity}" since:{start_date} until:{end_date}'

        # Scrape tweets using Snscrape
        tweet_df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(query).get_items(), num_tweets))

        # Clean up and rename columns
        tweet_df["entity"] = entity
        tweet_df = tweet_df[['entity', 'url', 'date', 'renderedContent', 'id', 'user', 'likeCount', 'retweetCount', 'quoteCount', 'viewCount']]
        tweet_df.rename(columns={'date': 'postedTime', 'renderedContent': 'tweetText'}, inplace=True)

        # Extract the username from the user column and drop the original user column
        tweet_df.loc[:, "username"] = tweet_df.loc[:, "user"].apply(lambda x: x["username"])
        tweet_df.drop(['user'], axis=1, inplace=True)

        # Add columns for the scraped time, date, month, and year
        jakarta_timezone = pytz.timezone('Asia/Jakarta')
        current_time = datetime.datetime.now(tz=jakarta_timezone)
        tweet_df['scrapedTime'] = pd.to_datetime(current_time)
        tweet_df['time'] = tweet_df['scrapedTime'].dt.strftime('%H:%M')
        tweet_df['date'] = tweet_df['scrapedTime'].dt.date
        tweet_df['month'] = tweet_df['scrapedTime'].dt.month
        tweet_df['year'] = tweet_df['scrapedTime'].dt.year

        # Concat the tweets for this entity to the overall DataFrame
        all_tweets_df = pd.concat([all_tweets_df, tweet_df], ignore_index=True)

    return all_tweets_df


# List of top Indonesian politicians
politicians = ["Prabowo", "Puan Maharani", "Anies Baswedan", "Jokowi", "Pemilu", "Ganjar"]

# Get dates two days before today and today
today = datetime.date.today()
two_days_before = today - datetime.timedelta(days=2)

# Call the function with the updated list and dates
politician_tweets_df = get_tweets(politicians, two_days_before, today, 10)
display(politician_tweets_df, politician_tweets_df.shape)


In [18]:
with open("../cleaned-article.txt") as f:
    article = f.read()

texts = text_splitter.split_text(article)

In [28]:
docs = [Document(page_content=t) for t in texts[:1]]
docs

[Document(page_content='Arman terperanjat saat ditawari endorsement yang nilainya delapan kali lipat dari yang biasa ia terima. Tawaran itu datang dari rumah judi online. Jika biasanya ia dibayar Rp1 juta untuk satu postingan, ia ditawari Rp8 juta. Tawaran itu jelas menggiurkan. Arman bukan nama sebenarnya. Usia 24, domisili Semarang. Pengikutnya di Instagram 300 ribuan. Ia rutin mengunggah konten-konten seputar esport: mulai dari panduan mekanik untuk hero dalam permainan MOBA (Multiplayer Online Battle Arena), tips dan trik, sampai trivia tim-tim esport yang berkompetisi di liga lokal. Selain itu, Arman juga streamer yang kontennya diunggah di YouTube.\n\nEndorsement yang sebelumnya pernah ia ambil antara lain dari gerai top-up gim daring dan produk digital, brand pakaian, minuman kemasan saset, dan dagangan temannya. Rumah judi online itu meminta Arman mempromosikan permainan slot di situsweb mereka. Rinciannya, Rp8 juta untuk satu postingan di feed dengan format bebas (foto atau vi

In [21]:
chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(docs)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


'Influencers and celebrities in Indonesia are being offered large sums of money to promote online gambling sites on their social media accounts. The practice is common among online gambling houses, who target those with large followings on Instagram and YouTube. The celebrities often use euphemisms such as "game online" instead of "gambling" and claim that the sites are licensed and accredited. However, the industry is largely unregulated and has been linked to money laundering, drug and human trafficking, and prostitution. The Global Initiative Against Transnational Organized Crime has identified the Asia-Pacific region as the largest and fastest-growing gambling market in the world. The Indonesian government has taken action against those promoting online gambling, with penalties including up to 10 years in prison and fines of up to IDR 1 billion ($70,000).'

In [30]:
prompt_template = """ Write a concise summary of the following article:

{text}

CONCISE SUMMARY IN BAHASA INDONESIA:
"""

myprompt = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(OpenAI(temperature=0, openai_api_key=api_key, model_name="gpt-3.5-turbo"), 
                             chain_type="map_reduce", return_intermediate_steps=True,
                             map_prompt=myprompt, combine_prompt=myprompt)

chain({"input_documents": docs}, return_only_outputs=True)

{'intermediate_steps': ['Sebuah artikel di Tirto.id membahas tentang praktik endorsement oleh rumah judi online kepada selebgram dan artis di Indonesia. Beberapa selebgram dan artis telah menerima tawaran endorsement dari rumah judi online dengan bayaran yang jauh lebih besar dari biasanya. Mereka diminta untuk mempromosikan permainan slot di situs web judi online tersebut. Beberapa di antaranya adalah Nikita Mirzani, Amanda Manopo, Chelsea Veronnia, Sheila Marcia, Dewi Persik, Denny Cagur, Ari Lasso, dan Boy William. Mereka mengganti frasa "judi online" dengan "game online" dan mengulang-ulang kalimat "game online ini sudah mempunyai lisensi dan terakreditasi". Namun, praktik ini ilegal dan berisiko bagi selebgram dan artis yang terlibat. Beberapa di antaranya telah dilaporkan ke kepolisian.'],
 'output_text': 'Beberapa selebgram dan artis di Indonesia telah menerima tawaran endorsement dari rumah judi online dengan bayaran yang jauh lebih besar dari biasanya untuk mempromosikan perma